(orc-chp-label)=

# ORC Combined Heat and Power

## Introduction

Einführungstext

## Excercise 1

Build a power plant working combined heat and power. The heat source is a geothermal field.

- Erstelle ein Modell und simuliere dieses in TESPy
- Beide Modelle aufbauen

## Proposed solution 1.1

**First Step**: Build the model topology of geothermal combined heat and power plan

In [ ]:
from tespy.networks import Network
from tespy.connections import Connection, Bus
from tespy.components import (
    CycleCloser, Source, Sink, Pump, Turbine, HeatExchanger,HeatExchangerSimple,
    Condenser, Drum, DropletSeparator, Merge
    )

# Network
nw = Network(fluids=['water', 'air', 'Isopentane'])
nw.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

flu_vec_orc = {'water': 0, 'air': 0, 'Isopentane': 1}
flu_vec_water = {'water': 1, 'air': 0, 'Isopentane': 0}
flu_vec_air = {'water': 0, 'air': 1, 'Isopentane': 0}

# Components
# ORC cycle with condenser coolant cycle
orc_cc = CycleCloser('ORC Cycle Closer')
orc_preheater = HeatExchanger('ORC Preheater')
orc_drum = Drum('ORC Drum')
orc_evaporator = HeatExchanger('ORC Evaporator')
orc_turbine = Turbine('ORC Turbine')
orc_condenser = Condenser('ORC Condenser')
cool_source = Source('Coolant Source')
cool_sink = Sink('Coolant Sink')
orc_pump = Pump('ORC Pump')

# Steam cycle with district heating system
steam_turbine = Turbine('Steam Turbine')
dh_condenser = Condenser('District Heating Condenser')
dh_preheater = HeatExchanger('District Heating Preheater')
dh_cc = CycleCloser('District Heating Cycle Closer')
dh_pump = Pump('District Heating Pump')
dh_heatsink = HeatExchangerSimple('District Heating Consumer')
steam_pump = Pump('Steam Pump')

# Geothermal heat source with interfaces
geo_source = Source('Geothermal Source')
seperator = DropletSeparator('Seperator')
merge = Merge('Merge')
geo_pump = Pump('Geothermal Pump')
geo_sink = Sink('Geothermal Sink')

# Connections
# ORC cycle
orc_pump2orc_cc = Connection(orc_pump, 'out1', orc_cc, 'in1', label='0')
orc_cc2orc_preheater = Connection(orc_cc, 'out1', orc_preheater, 'in2', label='1')
orc_preheat2orc_drum = Connection(orc_preheater, 'out2', orc_drum, 'in1', label='2')
orc_drum2orc_eva = Connection(orc_drum, 'out1', orc_evaporator, 'in2', label='3')
orc_eva2orc_drum = Connection(orc_evaporator, 'out2', orc_drum, 'in2', label='4')
orc_drum2orc_turb = Connection(orc_drum, 'out2', orc_turbine, 'in1', label='5')
orc_turb2orc_cond = Connection(orc_turbine, 'out1', orc_condenser, 'in1', label='6')
orc_cond2orc_pump = Connection(orc_condenser, 'out1', orc_pump, 'in1', label='7')

nw.add_conns(
    orc_pump2orc_cc, orc_cc2orc_preheater, orc_preheat2orc_drum, orc_drum2orc_eva, 
    orc_eva2orc_drum, orc_drum2orc_turb, orc_turb2orc_cond, orc_cond2orc_pump
    )

# Condenser coolant cycle
cool_source2orc_cond = Connection(cool_source, 'out1', orc_condenser, 'in2', label='41')
orc_cond2cool_sink = Connection(orc_condenser, 'out2', cool_sink, 'in1', label='42')

nw.add_conns(cool_source2orc_cond, orc_cond2cool_sink)

# Steam cycle
sep2steam_turb = Connection(seperator, 'out1', steam_turbine, 'in1', label='11')
steam_turb2dh_cond = Connection(steam_turbine, 'out1', dh_condenser, 'in1', label='12')
dh_cond2dh_preheat = Connection(dh_condenser, 'out1', dh_preheater, 'in1', label='13')
dh_preheat2steam_pump = Connection(dh_preheater, 'out1', steam_pump, 'in1', label='14')
steam_pump2merge = Connection(steam_pump, 'out1', merge, 'in1', label='15')

nw.add_conns(
    sep2steam_turb, steam_turb2dh_cond, dh_cond2dh_preheat, dh_preheat2steam_pump, 
    steam_pump2merge
    )

# District heating system
dh_heatsink2dh_cc = Connection(dh_heatsink, 'out1', dh_cc, 'in1', label='30')
dh_cc2dh_pump = Connection(dh_cc, 'out1', dh_pump, 'in1', label='31')
dh_pump2dh_preheat = Connection(dh_pump, 'out1', dh_preheater, 'in2', label='32')
dh_preheat2dh_cond = Connection(dh_preheater, 'out2', dh_condenser, 'in2', label='33')
dh_cond2dh_heatsink = Connection(dh_condenser, 'out2', dh_heatsink, 'in1', label='34')

nw.add_conns(
    dh_heatsink2dh_cc, dh_cc2dh_pump, dh_pump2dh_preheat, dh_preheat2dh_cond,
    dh_cond2dh_heatsink
    )

# Geothermal Heat Source
geo_source2sep = Connection(geo_source,'out1', seperator, 'in1', label='21')
sep2orc_eva = Connection(seperator,'out2', orc_evaporator, 'in1', label='22')
orc_eva2orc_preheat = Connection(orc_evaporator,'out1', orc_preheater, 'in1', label='23')
orc_preheat2merge = Connection(orc_preheater,'out1', merge, 'in2', label='24')
merge2geo_pump = Connection(merge,'out1', geo_pump, 'in1', label='25')
geo_pump2geo_sink = Connection(geo_pump,'out1', geo_sink, 'in1', label='26')

nw.add_conns(
    geo_source2sep, sep2orc_eva, orc_eva2orc_preheat, orc_preheat2merge,
    merge2geo_pump, geo_pump2geo_sink
    )

**Second Step**:  Set the process parameters and starting values and define the energy busses

In [ ]:
from CoolProp.CoolProp import PropsSI as PSI

# Component Parametrization
orc_turbine.set_attr(eta_s=0.9)
steam_turbine.set_attr(eta_s=0.9)

steam_pump.set_attr(eta_s=0.8)
orc_pump.set_attr(eta_s=0.8)
geo_pump.set_attr(eta_s=0.8)
dh_pump.set_attr(eta_s=0.8)

orc_preheater.set_attr(pr1=1, pr2=1)
orc_evaporator.set_attr(pr1=1, pr2=1)
orc_condenser.set_attr(pr1=1, pr2=1)
dh_condenser.set_attr(pr1=1, pr2=1)
dh_preheater.set_attr(pr1=1, pr2=1)
dh_heatsink.set_attr(pr=1)

# Connection Parametrization
# Geothermal Heat source
geo_source2sep.set_attr(m=500, p=10.8, x=0.45, fluid=flu_vec_water)
geo_pump2geo_sink.set_attr(T=55, p=30)

# District Heating Network
dh_cond2dh_heatsink.set_attr(T=100, p=10, fluid=flu_vec_water)
dh_heatsink2dh_cc.set_attr(T=50)

# Cooling Cycle
cool_source2orc_cond.set_attr(T=15, p=1.013)

# Starting Values

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps(
    {'comp': geo_source, 'base': 'bus'},
    {'comp': geo_sink, 'base': 'component'}
    )

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': steam_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': orc_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': steam_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': orc_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': geo_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': dh_pump, 'char': 0.975*0.99, 'base': 'bus'}
    )

heat_out_dh = Bus('Heat Output District Heating')
heat_out_dh.add_comps({'comp': dh_heatsink, 'base': 'component'})

heat_out_cool = Bus('Heat Output Cooling')
heat_out_cool.add_comps(
    {'comp': cool_source, 'base': 'bus'},
    {'comp': cool_sink, 'base': 'component'}
    )

nw.add_busses(heat_in, power_out, heat_out_dh, heat_out_cool)

**Third step**: Solve model with starting and final values

In [ ]:
# Solve model
nw.set_attr(iterinfo=False)
nw.solve(mode='design')
# eta_th = abs(power_out.P.val)/heat_in.P.val
# print(f'eta_th = {eta_th:.3f}')

## Proposed solution 1.2

**First Step**: Build the model topology of geothermal power plant

**Second Step**:  Set the process parameters and starting values and define the energy busses

**Third step**: Solve model with starting and final values

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Excercise 2

- Energetische

- Treffe eine Auswahl von geeigneten Arbeitsmedien 
- Auswahl und Begründung der Nutzung thermodynamischer Stoffdatenmodelle für die Simulation.

In [ ]:
# code of solution

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Excercise 3

- Definition und Nutzung geeigneter Größen zur Bewertung der Prozessentwürfe
- anschließender Vergleich sowie Diskussion.

In [ ]:
# code of solution

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Lessons Learned

- Lessons Learned 1
- Lessons Learned 2
- Lessons Learned 3